# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
keys:列表，列表元素表示的是进行级联的df的一个名称
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

In [1]:
import numpy as np
from pandas import Series,DataFrame
import pandas as pd

### 1)匹配级联

In [2]:
df1 = DataFrame(data=np.random.randint(0,100,size=(3,3)),index=['a','b','c'],columns=['A','B','C'])
df1

,A,B,C
a,34,81,29
b,53,87,37
c,50,73,57


In [7]:
pd.concat([df1,df1],axis=0)

,A,B,C
a,34,81,29
b,53,87,37
c,50,73,57
a,34,81,29
b,53,87,37
c,50,73,57


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有2种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

In [14]:
df2 = DataFrame(data=np.random.randint(0,100,size=(3,3)),index=['a','c','d'],columns=['A','C','D'])
df2

,A,C,D
a,35,14,34
c,7,39,85
d,79,1,70


In [16]:
pd.concat([df1,df2],axis=1,join="outer")

/Users/wangjifei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,A,C,D
a,34.0,81.0,29.0,35.0,14.0,34.0
b,53.0,87.0,37.0,NaN,NaN,NaN
c,50.0,73.0,57.0,7.0,39.0,85.0
d,NaN,NaN,NaN,79.0,1.0,70.0


In [17]:
pd.concat([df1,df2],axis=0,join="inner")

,A,C
a,34,29
b,53,37
c,50,57
a,35,14
c,7,39
d,79,1


In [18]:
pd.concat([df1,df2],axis=1,join="inner",keys=['AAA','BBB'])

AAA         BBB        
    A   B   C   A   C   D
a  34  81  29  35  14  34
c  50  73  57   7  39  85

In [9]:
pd.concat([df1,df2],axis=0,join="inner",keys=['AAA','BBB'],ignore_index=True)

,A,C
0,66,0
1,70,11
2,64,75
3,64,31
4,74,65
5,87,0


### 3) 使用df.append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [10]:
df1.append(df2)

,A,B,C,D
a,66,51.0,0,NaN
b,70,63.0,11,NaN
c,64,76.0,75,NaN
a,64,NaN,31,18.0
c,74,NaN,65,62.0
d,87,NaN,0,83.0


In [22]:
df2

,A,C,D
a,35,14,34
c,7,39,85
d,79,1,70


## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

参数：
- how：out取并集   inner取交集

- on：当有多列相同的时候，可以使用on来指定使用那一列进行合并，on的值为一个列表

###  1) 一对一合并

In [24]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],'group':['Accounting','Engineering','Engineering'],})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [25]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],'hire_date':[2004,2008,2012],})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [26]:
pd.merge(df1,df2)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


### 2) 多对一合并

In [15]:
df3 = DataFrame({'employee':['Lisa','Jake'],'group':['Accounting','Engineering'],'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [16]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],'supervisor':['Carly','Guido','Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [17]:
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


### 3) 多对多合并

In [20]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [21]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [22]:
pd.merge(df1,df5)

,employee,group,supervisor
0,Jake,Engineering,Carly
1,Jake,Engineering,Guido
2,Lisa,Engineering,Carly
3,Lisa,Engineering,Guido


In [23]:
pd.merge(df1,df5,how='outer')

,employee,group,supervisor
0,Bob,Accounting,NaN
1,Jake,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Carly
4,Lisa,Engineering,Guido
5,NaN,HR,Steve


In [24]:
pd.merge(df1,df5,how='left')

,employee,group,supervisor
0,Bob,Accounting,NaN
1,Jake,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Carly
4,Lisa,Engineering,Guido


- 加载excl数据:pd.read_excel('excl_path',sheetname=1)

In [25]:
df1 = pd.read_excel('./data.xlsx',sheetname=1)
df1

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [26]:
df2 = pd.read_excel('./data.xlsx',sheetname=2)
df2

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [27]:
df3 = pd.read_excel('./data.xlsx',sheetname=3)
df3

,经销商,发货地区,手机型号
0,dancer,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


In [28]:
df4 = pd.read_excel('./data.xlsx',sheetname=4)
df4

,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


In [30]:
pd.merge(df3,df4,on="手机型号")

,经销商,发货地区_x,手机型号,发货地区_y,价格
0,dancer,beijing,iPhone,beijing,7000
1,dancer,beijing,iPhone,guangzhou,7600
2,dancer,beijing,iPhone,shenzhen,7400
3,tom,guangzhou,iPhone,beijing,7000
4,tom,guangzhou,iPhone,guangzhou,7600
5,tom,guangzhou,iPhone,shenzhen,7400
6,lucy,beijing,Android,beijing,3600
7,lucy,beijing,Android,guangzhou,4200
8,lucy,beijing,Android,shenzhen,3900
9,mery,guangzhou,Android,beijing,3600


### 4) key的规范化

- 当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

In [32]:
df1 = DataFrame({'employee':['Jack',"Summer","Steve"],
                 'group':['Accounting','Finance','Marketing']})
df1

,employee,group
0,Jack,Accounting
1,Summer,Finance
2,Steve,Marketing


In [33]:
df2 = DataFrame({'employee':['Jack','Bob',"Jake"],
                 'hire_date':[2003,2009,2012],
                'group':['Accounting','sell','ceo']})
df2

,employee,group,hire_date
0,Jack,Accounting,2003
1,Bob,sell,2009
2,Jake,ceo,2012


In [35]:
pd.merge(df1,df2,on="group",suffixes=('_甲','乙'),how="outer")

,employee_甲,group,employee乙,hire_date
0,Jack,Accounting,Jack,2003.0
1,Summer,Finance,NaN,NaN
2,Steve,Marketing,NaN,NaN
3,NaN,sell,Bob,2009.0
4,NaN,ceo,Jake,2012.0


   - 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [36]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})
df1

,employee,group,hire_date
0,Bobs,Accounting,1998
1,Linda,Product,2017
2,Bill,Marketing,2018


In [38]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})
df5

,hire_dates,name
0,1998,Lisa
1,2016,Bobs
2,2007,Bill


In [39]:
pd.merge(df1,df5,left_on="employee",right_on='name')

,employee,group,hire_date,hire_dates,name
0,Bobs,Accounting,1998,2016,Bobs
1,Bill,Marketing,2018,2007,Bill


============================================

练习：


1. 自行练习多对一，多对多的情况  

2. 自学left_index,right_index

============================================

### 5) 内合并与外合并:out取并集   inner取交集

- 内合并：只保留两者都有的key（默认模式）

In [14]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})


- 外合并 how='outer'：补NaN

In [15]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})


## 作业
## 3. 案例分析：美国各州人口数据分析